
# Azure Machine Learning による設備メンテナンス

## Workspace 設定

In [3]:
from azureml.core import Workspace, Experiment
from azureml.train.automl import AutoMLConfig
import logging

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

experiment = Experiment(workspace = ws, name = "automl-classification-pm")

Found the config file in: /Users/konabuta/Project/AzurePytorch/aml_config/config.json
Workspace name: azureml
Azure region: southeastasia
Subscription id: 9c0f91b8-eb2f-484c-979c-15848c098a6b
Resource group: azureml_sea


## データ準備

In [4]:
import pandas as pd 

train_df = pd.read_csv('./data/train.csv', sep=",", header=0)
test_df = pd.read_csv('./data/test.csv', sep=",", header=0)

In [5]:
train_df.head(10)

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,RUL,label1,label2,cycle_norm
0,1,1.0,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,...,0.0,0.333333,0.0,0.0,0.713178,0.724662,191.0,0,0,0.000000
1,1,2.0,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,...,0.0,0.333333,0.0,0.0,0.666667,0.731014,190.0,0,0,0.002770
2,1,3.0,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,...,0.0,0.166667,0.0,0.0,0.627907,0.621375,189.0,0,0,0.005540
3,1,4.0,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,...,0.0,0.333333,0.0,0.0,0.573643,0.662386,188.0,0,0,0.008310
4,1,5.0,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,...,0.0,0.416667,0.0,0.0,0.589147,0.704502,187.0,0,0,0.011080
5,1,6.0,0.252874,0.416667,0.0,0.0,0.268072,0.292784,0.272113,0.0,...,0.0,0.250000,0.0,0.0,0.651163,0.652720,186.0,0,0,0.013850
6,1,7.0,0.557471,0.583333,0.0,0.0,0.382530,0.463920,0.261985,0.0,...,0.0,0.333333,0.0,0.0,0.744186,0.667219,185.0,0,0,0.016620
7,1,8.0,0.304598,0.750000,0.0,0.0,0.406627,0.259865,0.316003,0.0,...,0.0,0.250000,0.0,0.0,0.643411,0.574979,184.0,0,0,0.019391
8,1,9.0,0.545977,0.583333,0.0,0.0,0.274096,0.434707,0.211850,0.0,...,0.0,0.333333,0.0,0.0,0.705426,0.707539,183.0,0,0,0.022161
9,1,10.0,0.310345,0.583333,0.0,0.0,0.150602,0.440375,0.307394,0.0,...,0.0,0.416667,0.0,0.0,0.627907,0.794256,182.0,0,0,0.024931


In [6]:
train_df.tail(10)

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,RUL,label1,label2,cycle_norm
20621,100,191.0,0.471264,0.500000,0.0,0.0,0.746988,0.868324,0.758609,0.0,...,0.0,0.833333,0.0,0.0,0.193798,0.314278,9.0,1,2,0.526316
20622,100,192.0,0.448276,0.583333,0.0,0.0,0.698795,0.658164,0.628460,0.0,...,0.0,0.750000,0.0,0.0,0.325581,0.252416,8.0,1,2,0.529086
20623,100,193.0,0.494253,0.666667,0.0,0.0,0.566265,0.627207,0.787981,0.0,...,0.0,0.750000,0.0,0.0,0.255814,0.177851,7.0,1,2,0.531856
20624,100,194.0,0.436782,0.750000,0.0,0.0,0.756024,0.572269,0.762323,0.0,...,0.0,0.500000,0.0,0.0,0.186047,0.328915,6.0,1,2,0.534626
20625,100,195.0,0.488506,0.416667,0.0,0.0,0.662651,0.632221,0.838116,0.0,...,0.0,0.500000,0.0,0.0,0.000000,0.411627,5.0,1,2,0.537396
20626,100,196.0,0.477011,0.250000,0.0,0.0,0.686747,0.587312,0.782917,0.0,...,0.0,0.750000,0.0,0.0,0.271318,0.109500,4.0,1,2,0.540166
20627,100,197.0,0.408046,0.083333,0.0,0.0,0.701807,0.729453,0.866475,0.0,...,0.0,0.583333,0.0,0.0,0.124031,0.366197,3.0,1,2,0.542936
20628,100,198.0,0.522989,0.500000,0.0,0.0,0.665663,0.684979,0.775321,0.0,...,0.0,0.833333,0.0,0.0,0.232558,0.053991,2.0,1,2,0.545706
20629,100,199.0,0.436782,0.750000,0.0,0.0,0.608434,0.746021,0.747468,0.0,...,0.0,0.583333,0.0,0.0,0.116279,0.234466,1.0,1,2,0.548476
20630,100,200.0,0.316092,0.083333,0.0,0.0,0.795181,0.639634,0.842167,0.0,...,0.0,0.666667,0.0,0.0,0.178295,0.218172,0.0,1,2,0.551247


In [7]:
#　特徴量となる列名
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle']
sequence_cols.extend(sensor_cols)
print(sequence_cols)

['setting1', 'setting2', 'setting3', 'cycle', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']


## Automated ML 設定

In [8]:
automl_config = AutoMLConfig(task = 'classification',
                             iteration_timeout_minutes = 10,
                             iterations = 10,
                             primary_metric = 'AUC_weighted',
                             n_cross_validations = 2,
                             debug_log = 'automl.log',
                             verbosity = logging.INFO,
                             X = train_df[sequence_cols], 
                             y = train_df['label1'].values
                             )

## 実行

In [9]:
%%time
local_run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_e11f6e55-767f-4fc5-90ef-13fd4a1c6d3a
*******************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
TRAINFRAC: Fraction of the training data to train on.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
*******************************************************************************************************

 ITERATION   PIPELINE                                       TRAINFRAC  DURATION      METRIC      BEST
         0   MaxAbsScaler LightGBM                          1.0000     0:00:17       0.9886    0.9886
         1   StandardScalerWrapper LogisticRegression       1.0000     0:00:15       0.9895    0.9895
         2   RobustScaler GradientBoosting                  1.0000     0:00:29       0.988

In [ ]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

In [ ]:
local_run